In [ ]:
# fmt: off
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%load_ext jupyter_black
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mls_scf_tools.mls_pint import ureg
plt.style.use("mls_scf")
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('__')]


In [ ]:
import intervaltree
import itertools
import pint
from matplotlib import colormaps
import warnings
import json
import pathlib
from matplotlib.backends.backend_pdf import PdfPages
import re

In [ ]:
%autoreload 2
import njl_corf
from njl_corf.pyfcctab import ureg
import njl_corf.pyfcctab as pyfcctab
import njl_corf.pyoscar as pyoscar
import njl_corf.band_figure as band_figure
import njl_corf.wrc27_views as wrc27_views

In [ ]:
allocation_database = pyfcctab.read(skip_additionals=False)

In [ ]:
print(allocation_database.r1[35.51 * ureg.GHz][0].bounds)

In [ ]:
oscar_database = pyoscar.read()
oscar_database = pyoscar.merge_sensors(
    oscar_database,
    {
        "AMSR": "AMSR*",
        "ATMS": "ATMS*",
        "AWS": "AWS*",
        "MWTS": "MWTS*",
        "SSMIS": "SSMIS*",
        "AMSU": "AMSU*",
        #        "MWRI": "MWRI*",
        "MWS": "MWS*",
        "MTVZA": "MTVZA*",
        "MWI": "MWI*",
        "SRAL": "SRAL*",
        "OSCAT": "OSCAT*",
        "Altimetry": ["ALT*", "RA*", "Altimeter*"],
        "Precipitation": ["PR*", "DPR*", "Rainradar"],
        "Poseidon": "Poseidon*",
        "SWIM": "SWIM*",
        "MWR": "MWR*",
        "WSF": "WSF*",
        "MLS": "MLS*",
        "ICI": "ICI*",
    },
)

In [ ]:
config = band_figure.FigureConfiguration.load()

In [ ]:
collation_rules = {
    "Space research": ["Space research*"],
    "Mobile": ["Mobile", "Mobile*"],
    "Fixed": ["Fixed", "Fixed*"],
    "Aeronautical": ["Aeronautical*"],
    "EESS (passive)": ["Earth exploration-satellite *(passive)"],
    "EESS (active)": ["Earth exploration-satellite *(active)"],
    "EESS (comms)": ["Earth exploration-satellite *(*to*)"],
    "Amateur": ["Amateur*"],
    "Radionavigation": ["Radionavigation*"],
    "Meteorological": ["Meteorological*"],
    "Space operation": ["Space operation*"],
    "Radiodetermination": ["Radiodetermination*"],
    "Radio astronomy": ["Radio Astronomy*"],
}

In [ ]:
ai_ranges = wrc27_views.get_ai_info()

In [ ]:
def all_wrc27_figures():
    config = band_figure.FigureConfiguration.load()
    ureg.setup_matplotlib()
    with PdfPages("example-ai-figures.pdf") as pdf:
        for range_name, range_details in ai_ranges.items():
            print(f"{range_name}: {range_details}")
            if range_details.frequency_band.start is None:
                continue
            fig, ax = plt.subplots(figsize=[8.5, 11], layout="constrained")
            band_figure.views_plot(
                frequency_range=range_details,
                allocation_database=allocation_database,
                oscar_database=oscar_database,
                ax=ax,
                figure_configuration=config,
                collation_rules={},  # collation_rules,
                omit_band_borders=True,
            )
            ax.set_title(range_name)
            pdf.savefig(fig)
            plt.close()


all_wrc27_figures()

In [ ]:
test = band_figure.views_plot(
    frequency_range=slice(1300 * ureg.MHz, 1450 * ureg.MHz),
    allocation_database=allocation_database,
    oscar_database=oscar_database,
    figure_configuration=config,
    collation_rules=collation_rules,
    frequency_margin=200 * ureg.MHz,
)

In [ ]:
allocation_database.itu.find_closest_matching_band(
    frequency=1475 * ureg.MHz,
    direction=1,
    condition=lambda band: band.has_allocation("Radio Astronomy"),
)

In [ ]:
def make_html():
    for ai in ai_ranges.keys():
        wrc27_views.ai_html_summary(
            ai,
            allocation_database=allocation_database,
            oscar_database=oscar_database,
        )


make_html()

In [ ]:
wrc27_views.generate_index_html()

In [ ]:
import pathlib
import shutil

In [ ]:
# wrc27_views.push_information()